In [ ]:
! pip install -U bert-serving-server bert-serving-client
# https://github.com/mtala3t/Identify-the-Sentiments-AV-NLP-Contest

     |████████████████████████████████| 61 kB 268 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=e9e7020a6f724ebd9eb4b25477e611707726111d6132d0f1f3b5d8ad27e30a74
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


https://datahack.analyticsvidhya.com/contest/linguipedia-codefest-natural-language-processing-1/#About

https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
import pickle
import logging
from bert_serving.client import BertClient

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_colwidth', 200)


In [ ]:
# read data
train = pd.read_csv("/content/drive/My Drive/sentiment_ELMo/train_2kmZucJ.csv")
test = pd.read_csv("/content/drive/My Drive/sentiment_ELMo/test_oJQbWVk.csv")


In [ ]:
print (train.shape, test.shape)
print (train['label'].value_counts())
print (train.head())


(7920, 3) (1953, 2)
0    5894
1    2026
Name: label, dtype: int64
   id  label  \
0   1      0   
1   2      0   
2   3      0   
3   4      0   
4   5      1   

                                                                                                                                 tweet  
0     #fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone  
1  Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/  
2          We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu  
3                     I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/  
4         What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!  


In [ ]:
# data cleaning: remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [ ]:
# remove twitter handles (@user)
train['clean_tweet'] = train['clean_tweet'].apply(lambda x: re.sub("@[\w]*", '', x))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: re.sub("@[\w]*", '', x))
  

In [ ]:
# remove punctuation marks
punctuation = '.,\'!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))


In [ ]:
# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

In [ ]:
# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [ ]:
#Normalize the words to its base form
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])


In [ ]:
# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [ ]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])


In [ ]:
# Extract BERT embeddings function
def bert_vectors(x):
  
    # make a connection with the BERT server using it's ip address
    bc = BertClient()
    
    return bc.encode(x.tolist())


In [ ]:
# Extract BERT embeddings
bert_train = bert_vectors(train['clean_tweet'])
bert_test = bert_vectors(test['clean_tweet'])


In [ ]:
# save bert_train_new
pickle_out = open("bert_train_03032019.pickle","wb")
pickle.dump(bert_train, pickle_out)
pickle_out.close()

In [ ]:
# save bert_test_new
pickle_out = open("bert_test_03032019.pickle","wb")
pickle.dump(bert_test, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("bert_train_03032019.pickle", "rb")
bert_train_new = pickle.load(pickle_in)


In [ ]:
# load elmo_train_new
pickle_in = open("bert_test_03032019.pickle", "rb")
bert_test_new = pickle.load(pickle_in)


In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(bert_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)
print (ytrain.shape, yvalid.shape)

In [ ]:
lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(xvalid)
print (f1_score(yvalid, preds_valid))

In [ ]:
# make predictions on test set
preds_test = lreg.predict(bert_test_new)

In [ ]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg_bert.csv", index=False)